In [1]:
!pip install langchain langchain-huggingface huggingface_hub

In [13]:
from google.colab import userdata
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')


# StrOutput parser
 It takes the raw output from the language model and returns it as a plain string, without making any changes or trying to structure it.

In [3]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2',temperature=0.7)
model = ChatHuggingFace(llm=llm )

prompt_1 = PromptTemplate(
    template="Write a details report on {topic}",
    input_variables=["topic"],
)
prompt_2 = PromptTemplate(
    template="summarize this {text} in 5 line",
    input_variables=["text"],
)
parser = StrOutputParser()

chain = prompt_1 | model | parser | prompt_2 | model | parser

chain.invoke({"topic":"Attention machanism"})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


' Attention mechanisms are a vital component in deep learning, particularly for NLP and computer vision, allowing models to focus on relevant input data and improve performance. Self-attention, used in the Transformer model, enables each position in the sequence to attend to all others. Other types include dot-product, multi-head, and scaled dot-product attention. Attention mechanisms have led to state-of-the-art results in NLP tasks.'

# JsonOutputPerser

Parse the output of an LLM call to a JSON object.

In [4]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2',temperature=0.5)
model =ChatHuggingFace(llm=llm )

parser = JsonOutputParser()
prompt_1 = PromptTemplate(
    template ='write a 5 fact about {topic} /n {format_instructions}',
    input_variables=["topic"],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)
chain =prompt_1 | model | parser

chain.invoke({"topic":"Earth"})

{'fact1': 'The Earth is the third planet from the Sun and the fifth largest among the eight planets in the Solar System.',
 'fact2': "The Earth's surface is approximately 510 million square kilometers (197 million square miles). About 71% of it is covered by water and the remaining 29% by land.",
 'fact3': 'The interior of the Earth is divided into three main layers: the outer crust, the mantle, and the inner core. The inner core is the hottest place in the Solar System, with temperatures estimated to be around 5,500 degrees Celsius (9,932 degrees Fahrenheit).',
 'fact4': "The Earth's Axial Tilt is responsible for the seasons. The tilt is 23.5 degrees relative to the ecliptic: this is one reason for the seasons, as the amount of the Sun's rays reaching the Earth varies throughout the year.",
 'fact5': "The Earth's atmosphere is primarily composed of nitrogen (78%) and oxygen (21%), with traces of water vapor, carbon dioxide, and other gases. The atmosphere shields us from solar radiati

# StructuredOutputParser
The StructuredOutputParser in LangChain is a powerful tool used to parse the raw text output from a Large Language Model (LLM) into a predefined, structured format

In [8]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2',temperature=0.5)
model =ChatHuggingFace(llm=llm )


parser =StructuredOutputParser.from_response_schemas([
    ResponseSchema(name='book name',description='what is the name of the book'),
    ResponseSchema(name='auther',description='who is auther of the book'),
    ResponseSchema(name='genra',description='what is the genra of the book')
])

prompt = PromptTemplate(
    template = 'Give me fllowing information of this {name} book _ book name ,auther ,genra /n {format_instructions} ',
    input_variables=["name"],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

chain = prompt | model | parser

chain.invoke({"name":'herry potter'})

{'book name': "Harry Potter and the Philosopher's Stone",
 'auther': 'J.K. Rowling',
 'genra': 'Fantasy'}

# PydanticOutputParser
Parse an output using a pydantic model.



In [9]:
!pip install pydantic

In [12]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.5)
model = ChatHuggingFace(llm=llm)

# Define the Pydantic model with corrections
class Book(BaseModel):
    book_name: str = Field(description='what is the name of the book')
    author: str = Field(description='who is the author of the book')
    genre: str = Field(description='what is the genre of the book')
    hero: str = Field(description='who is the hero of the book')
    villain: str = Field(description='who is the villain of the book')
    main_character: List[str] = Field(description='who are the main characters of the book')

parser = PydanticOutputParser(pydantic_object=Book)

prompt = PromptTemplate(
    template="""Extract the following information about the book "{name}":
- Book Name
- Author
- Genre
- Hero
- Villain
- Main Characters (as a list)

{format_instructions}

Book: {name}
""",
    input_variables=["name"],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

chain = prompt | model | parser


try:
    result = chain.invoke({"name": "Harry Potter and the Philosopher's Stone"})
    print("Parsing successful!")
    print(result.model_dump_json(indent=2))
except Exception as e:
    print(f"An error occurred during invocation or parsing: {e}")


Parsing successful!
{
  "book_name": "Harry Potter and the Philosopher's Stone",
  "author": "J.K. Rowling",
  "genre": "Fantasy",
  "hero": "Harry Potter",
  "villain": "Sorcerer's Stone (Metamorphoned into Quirrell) / Lord Voldemort",
  "main_character": [
    "Harry Potter",
    "Ron Weasley",
    "Hermione Granger",
    "Professor Albus Dumbledore",
    "Professor Severus Snape",
    "Professor Quirinus Quirrell",
    "Ron's Brothers (Arthur, Fred, George, Percy)",
    "Tom Riddle (Flashbacks)",
    "Arthur Weasley",
    "Neville Longbottom",
    "Gilderoy Lockhart",
    "Hagrid",
    "Ginny Weasley",
    "Fawkes (Theophilus Nigel)",
    "Nearly Headless Nick",
    "Moaning Myrtle",
    "Professor Minerva McGonagall",
    "Professor Pomona Sprout",
    "Professor Filius Flitwick"
  ]
}
